In [77]:
import pyTigerGraph as tg
import pandas as pd
import gradio as gr
hostName = "https://4da1bd47ec034979919364bf0615f5f4.i.tgcloud.io"
graphName = "VWG"
secret = "uoabr03s0ls2prn9hpdupch6vquvn6gu"
userName = "user_1"
password = "A1z2e3r4*"
tigergraph_insights_map = "https://tools.tgcloud.io/insights/app/67eZLiuqbdk5nNg9QxWQfA/page/gvxC91DiSnRKvPvWzfR9iH/widgetShare/sQFKgLZh8iCgVFrarvUVTr?domain=4da1bd47ec034979919364bf0615f5f4.i&orgName=lis-gordias&clusterid=5fc3b84f-33fb-41b4-b856-2e983aa290f6&TigerGraphToken=3ff2a285-25b4-4a77-98c2-80bfbde3fd73"
graph = tg.TigerGraphConnection(host=hostName, graphname=graphName)

authToken = graph.getToken(secret)
authToken = authToken[0]
print(f"secret token: {authToken}")
conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password, apiToken=authToken)

secret token: uf9a4lta7oufn1ib110t49c4ig43pqe7


c:\Users\JulienRigot\Projets\zzEnvs\Transport\Lib\site-packages\pyTigerGraph\pyTigerGraphBase.py:105: DeprecationWarning: The `apiToken` parameter is deprecated; use `getToken()` function instead.
  warnings.warn(


# Tab 0 (cargar datos)

In [78]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
import networkx as nx

# Helper functions
def path_taken(results):
    edges = results[2]['@@display_edge_set']
    node_initial = None
    edge_to_ids = {edge['to_id'] for edge in edges}
    for edge in edges:
        if edge['from_id'] not in edge_to_ids:
            node_initial = edge['from_id']
            break
    if node_initial is None:
        raise ValueError("initial node cannot be determined.")
    current_node = node_initial
    path = [current_node]
    while True:
        next_node = None
        for edge in edges:
            if edge['from_id'] == current_node:
                next_node = edge['to_id']
                break
        if next_node is None:
            break
        path.append(next_node)
        current_node = next_node
    nodes = {node_name: node_name for node_name in path}
    ordered_nodes = [nodes[node_name] for node_name in path]
    return ordered_nodes

def add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch , charge):
    existing_cities = [city for city, _ in nodes_batch_full]
    for node, _ in nodes_batch:
        if node in existing_cities:
            # search if town already in nodes_batch_full
            for i, (city, stock) in enumerate(nodes_batch_full):
                if city == node:
                    # update stock
                    if i == 0:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] - charge})
                    else:
                        nodes_batch_full[i] = (city, {"Stock": nodes_batch_full[i][1]["Stock"] + charge})
            continue
        else:
            # add new entries to nodes_batch_full
            nodes_batch_full.extend([
                (nodes_batch[0][0], {"Stock": nodes_batch[0][1]["Stock"]}),
                (nodes_batch[1][0], {"Stock": nodes_batch[1][1]["Stock"]})
            ])
            return nodes_batch_full
    return nodes_batch_full

def add_or_update_final_edges_batch(edges_batch_full, edges_batch, charge):
    existing_edges = [(edge[0], edge[1]) for edge in edges_batch_full]
    
    for edge in edges_batch:
        # edge already exist ? 
        if (edge[0], edge[1]) in existing_edges or (edge[1], edge[0]) in existing_edges:
            # if yes, search if edge already in edges_batch_full
            for i, (origin, destination, movement) in enumerate(edges_batch_full):
                if (origin, destination) == (edge[0], edge[1]) or (origin, destination) == (edge[1], edge[0]):
                    # Update
                    edges_batch_full[i] = (origin, destination, {"Daily_movement": edges_batch_full[i][2]["Daily_movement"] + charge})
            continue
        else:
            # If no, add new entries to edges_batch_full
            edges_batch_full.append((edge[0], edge[1], {"Daily_movement": edge[2]["Daily_movement"]}))
    
    return edges_batch_full

# Main transfer function
def transfert_nodes_and_edges(node_initial, node_final, charge, weight_attribute="Capacity", nodetype="Nodes", edgetype="distribute_to"):
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full
    results_astar = conn.runInstalledQuery("tg_astar_test", params={
        "source_vertex": node_final, "source_vertex.type": nodetype,
        "target_vertex": node_initial, "target_vertex.type": nodetype,
        "e_type_set": edgetype, "weight_type": "FLOAT",
        "latitude": "latitude", "longitude": "longitude",
        "weight_attribute": weight_attribute,
        "print_stats": "True"
    })

    results_allpaths = conn.allPaths([("Nodes", node_initial)], [("Nodes", node_final)],maxLength=len(results_astar[2]["@@display_edge_set"])+1)


    G = nx.Graph()

    # add nodes
    for vertex in results_allpaths[0]['vertices']:
        G.add_node(vertex['v_id'], **vertex['attributes'])

    # add edges and attributes 
    for edge in results_allpaths[0]['edges']:
        G.add_edge(edge['from_id'], edge['to_id'], **edge['attributes'])

    
    def find_two_lowest_price_paths(G, node_initial, node_final, edge_len):
            
        all_paths = nx.all_simple_paths(G, node_initial, node_final)

        def calculate_total_price(G, path):
            total_price = 0
            for i in range(len(path) - 1):
                edge_data = G[path[i]][path[i + 1]]
                total_price += edge_data['Price']
            return total_price
        
        max_path_length = len(edge_len) + 2

        path_prices = []

        for path in all_paths:
            if len(path) <= max_path_length:
                total_price = calculate_total_price(G, path)
                path_prices.append((path, total_price))

        path_prices.sort(key=lambda x: x[1])

        top_two_paths = path_prices[:2]

        proposed_paths = []
    
        for path, total_price in top_two_paths:
            nodes_info = []
            edges_info = []

            for i in range(len(path) - 1):
                from_node_id = path[i]
                to_node_id = path[i + 1]
                edge_data = G[from_node_id][to_node_id]
                price = edge_data['Price']
                edges_info.append({'e_type': 'distribute_to', 'from_id': from_node_id, 'from_type': 'Nodes',
                                'to_id': to_node_id, 'to_type': 'Nodes', 'directed': False,
                                'attributes': {'Capacity': edge_data['Capacity'], 'Price': price,
                                                'Daily_movement': edge_data['Daily_movement']}})
                from_node_attributes = G.nodes[from_node_id]
                nodes_info.append({'v_id': from_node_id, 'v_type': 'Nodes', 'attributes': from_node_attributes})

            proposed_paths.append({'vertices': nodes_info, 'edges': edges_info, 'total_price': total_price})

        return proposed_paths
    
    top_alternative_paths_and_prices = find_two_lowest_price_paths(G, node_initial, node_final, results_astar[2]["@@display_edge_set"])


    order_taken = path_taken(results_astar)
    edge_set = results_astar[2]['@@display_edge_set']
    reordered_edges_set = []
    for location in order_taken:
        for edge in edge_set:
            if edge['from_id'] == location:
                reordered_edges_set.append(edge)

    node_set = results_astar[2]['tmp']
    reordered_nodes_set = []
    for location in order_taken:
        for item in node_set:
            if item['v_id'] == location:
                reordered_nodes_set.append(item)

    try:
        node_i_stock = reordered_nodes_set[0]["attributes"]["Stock"]
        node_f_stock = reordered_nodes_set[-1]["attributes"]["Stock"]
        node_i_unload_capacity = reordered_nodes_set[0]["attributes"]["UnloadCapacity"]
        node_f_load_capacity = reordered_nodes_set[-1]["attributes"]["LoadCapacity"]

        if charge > node_i_stock:
            error_message = f"{node_initial} to {node_final}: Not enough stock ({charge} > {node_i_stock})"
            log_list.append(error_message)

        if charge > node_f_load_capacity:
            error_message = f"{node_initial} to {node_final}: Not enough load capacity from receiving warehouse (charge: {charge} > LoadCapacity: {node_f_load_capacity})"
            log_list.append(error_message)

        if charge > node_i_unload_capacity:
            error_message = f"{node_initial} to {node_final}: Not enough unload capacity from starting warehouse (charge: {charge} > UnloadCapacity: {node_i_unload_capacity})"
            log_list.append(error_message)

        capacity_edge = reordered_edges_set[0]["attributes"]["Capacity"]
        daily_movement = reordered_edges_set[0]["attributes"]["Daily_movement"]

        if daily_movement + charge > capacity_edge:
            error_message = f"Send {capacity_edge} palets from {node_initial} to {node_final}: Not enough edge capacity ({daily_movement + charge} > {capacity_edge})"
            log_list.append(error_message)

        nodes_batch = [
            (node_initial, {"Stock": node_i_stock - charge}),
            (node_final, {"Stock": node_f_stock + charge})
        ]

        edges_batch = []
        for element in reordered_edges_set:
            element['attributes']['Daily_movement'] += charge
            edges_batch.append((element["from_id"], element["to_id"], {"Daily_movement": element["attributes"]["Daily_movement"]}))
            palets_cost += element["attributes"]["Price"]

        unique_origin_nodes.add(node_initial)
        unique_destination_nodes.add(node_final)
        operation_count += 1
        palets_number += charge

        return nodes_batch , edges_batch , top_alternative_paths_and_prices

    except Exception as e:
        error_message = f"Error in the transfer: {e}"
        log_list.append(error_message)
        return error_message

# Thread safety
lock = threading.Lock()

def make_daily_movements(input_file, column_origin, column_destination, column_transfert):
    global operation_count, palets_number, palets_cost, unique_origin_nodes, unique_destination_nodes, log_list, nodes_batch_full,alternatives_paths
    operation_count = 0
    palets_number = 0
    palets_cost = 0
    unique_origin_nodes = set()
    unique_destination_nodes = set()
    log_list = []
    nodes_batch_full = []
    edges_batch_full = []
    alternatives_paths = []

    df = pd.read_csv(input_file)

    def perform_transfer(row):
        try:
            origin = str(row[column_origin])
            destination = str(row[column_destination])
            volume = float(row[column_transfert])
            nodes_batch, edges_batch, top_alternative_paths_and_prices = transfert_nodes_and_edges(origin, destination, volume)
            with lock:
                add_or_update_final_nodes_batch(nodes_batch_full, nodes_batch, volume)
                add_or_update_final_edges_batch(edges_batch_full, edges_batch, volume)
                
                alternatives_paths.append(top_alternative_paths_and_prices)
        except Exception as e:
            log_list.append(f"{origin} to {destination}, no possible path exists.")

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = [executor.submit(perform_transfer, row) for index, row in df.iterrows()]
        for future in as_completed(futures):
            pass

    conn.upsertVertices("Nodes", nodes_batch_full)
    nodetype = "Nodes"
    edgetype = "distribute_to"

    conn.upsertEdges(sourceVertexType=nodetype, targetVertexType=nodetype, edgeType=edgetype, edges=edges_batch_full)

    return log_list

# Function tab 1 BIS (KPI daily movement)

In [79]:
def operation_number():
    return operation_count

def number_palets():
    return palets_number

def origin_number():
    global unique_origin_nodes
    return len(unique_origin_nodes)

def distinct_destinations():
    global unique_destination_nodes
    return len(unique_destination_nodes)

def total_cost():
    return palets_cost

def cost_per_palet():
    return palets_cost/palets_number

def get_KPIs_movements():
    return operation_number(), number_palets(), origin_number(), distinct_destinations(), total_cost(), cost_per_palet()

In [80]:
def alternative_path_1():
    path = []
    for edge in alternatives_paths[0][0]['edges']:
        path.append(edge['from_id'])
    path.append(alternatives_paths[0][0]['edges'][-1]['to_id'])

    return path

def alternative_price_1():
    return alternatives_paths[0][0]["total_price"]

def alternative_path_2():
    path = []
    for edge in alternatives_paths[0][1]['edges']:
        path.append(edge['from_id'])
    path.append(alternatives_paths[0][1]['edges'][-1]['to_id'])

def alternative_price_2():
    return alternatives_paths[0][1]["total_price"]

def get_KPIs_alternatives():
    return alternative_path_1(), alternative_price_1(), alternative_path_2(), alternative_price_2()

# Functions tab 1 ( KPI )

In [81]:
def get_edges(name_edge = "distribute_to"):
    numEdges = conn.getEdgeCount(name_edge)
    return numEdges

def get_nodes(name_node = "Nodes"):
    numNodes = conn.getVertexCount(name_node)
    return numNodes

def get_average_cost_per_edge(name_edge = "distribute_to"):
    edge = conn.getEdgeStats(name_edge)
    avg_capacity = edge[name_edge]['Capacity']['AVG']
    return avg_capacity

def get_total_capacity_edges(name_edge = "distribute_to"):
    edge = conn.getEdgesByType(name_edge)
    total_capacity_edges = 0
    for warehouse in edge :
        capacity = warehouse["attributes"]["Capacity"]     
        total_capacity_edges += capacity
    return total_capacity_edges

def get_total_capacity_warehouse(name_node = "Nodes"):
    almacen = conn.getVertices(name_node)
    total_capacity_warehouse = 0
    for warehouse in almacen :
        capacity = warehouse["attributes"]["Capacity"]      #REPLACE CAPACITY / CARGA REAL 
        total_capacity_warehouse += capacity
    return total_capacity_warehouse


def get_total_stock(node = "Nodes"):
    almacen = conn.getVertices(node)
    total_stock = 0
    for warehouse in almacen :
        stock = warehouse["attributes"]["Stock"]     
        total_stock += stock
    return total_stock


def get_average_capacity_nodes(edge = "distribute_to"):
    return get_total_capacity_warehouse()/conn.getEdgeCount(edge)

def get_KPIs_network():
    edges = get_edges()
    nodes = get_nodes()
    average_cost_edge = get_average_cost_per_edge()
    total_capacity_edge = get_total_capacity_edges()
    warehouse_total_capacity = get_total_capacity_warehouse()
    warehouse_total_stock = get_total_stock()
    return edges, nodes, average_cost_edge,total_capacity_edge,warehouse_total_capacity, warehouse_total_stock

# Functions tab 2 (modif red)

In [82]:
# conn.upsertEdge(sourceVertexType="Nodes",targetVertexType="Nodes",edgeType="distribute_to",sourceVertexId="Almacen 260",targetVertexId="Almacen 730",attributes={"Capacity" : 1000})

In [83]:
import json
global logs_modif_network
logs_modif_network = " "


# ADD NODES 

def add_node(id, attributes_dict=None, vertex_type="Nodes"):
    global logs_modif_network
    if attributes_dict is None:
        conn.upsertVertex(vertex_type, id)
        message =  f"Node {id} added"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn.upsertVertex(vertex_type, id, attributes=attributes)
        message =  f"Node {id} added with attributes {attributes} \n"
        logs_modif_network +=message
    return logs_modif_network

def get_values_nodes(*args):
    values = {key: val for key, val in zip(atributes_nodes.keys(), args)}
    return values

def get_values_edges(*args):
    values = {key: val for key, val in zip(atributes_edges.keys(), args)}
    return values


def add_node_with_values(id, *args):
    attributes = get_values_nodes(*args)
    attributes_json = json.dumps(attributes)
    return add_node(id, attributes_json)



# ADD EDGES 
def add_new_edge(source_vertex_id, target_vertex_id, attributes_dict=None,source_vertex_type="Nodes",target_vertex_type = "Nodes", edge_type="distribute_to"):
    global logs_modif_network
    if attributes_dict is None:
        conn.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id)
        message =  f"Edge from {source_vertex_type}:{source_vertex_id} to {target_vertex_type}:{target_vertex_id} of type {edge_type} added"
        logs_modif_network += message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn.upsertEdge(source_vertex_type, source_vertex_id, edge_type, target_vertex_type, target_vertex_id, attributes=attributes)
        message =  f"Edge from {source_vertex_type}:{source_vertex_id} to {target_vertex_type}:{target_vertex_id} of type {edge_type} added with attributes {attributes} \n"
        logs_modif_network += message
    return logs_modif_network

def add_edge_with_values(source_vertex_id, target_vertex_id, *args):
    attributes = get_values_edges(*args)
    attributes_json = json.dumps(attributes)
    return add_new_edge(source_vertex_id,target_vertex_id, attributes_json)


# GET ATTRIBUTES
attributes_nodes_list = []
attributes_edges_list = []
conex_nodes_list = conn.getVertexAttrs('Nodes')
conex_edges_list = conn.getEdgeAttrs('distribute_to')

for node in conex_nodes_list:
    attributes_nodes_list.append(node[0])

for edge in conex_edges_list:
    attributes_edges_list.append(edge[0])



# GET NODES NAMES
list_name_nodes = []
name_nodes_conn = conn.getVertices("Nodes")
for i in range(0, len(name_nodes_conn)):
    list_name_nodes.append(name_nodes_conn[i]["v_id"])

def add_edge(source_vertex_id, target_vertex_id, attributes_dict=None, vertex_type="Nodes", edgeType="distribute_to"):
    if attributes_dict is None:
        conn.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id)
        message =  f"Edge from {source_vertex_id} to {target_vertex_id} added"
        logs_modif_network +=message
        return logs_modif_network
    else:
        attributes = json.loads(attributes_dict)
        conn.upsertEdge(sourceVertexType=vertex_type,
                        targetVertexType=vertex_type,
                        edgeType=edgeType,
                        sourceVertexId=source_vertex_id,
                        targetVertexId=target_vertex_id,
                        attributes=attributes)
        message =  f"Edge from {source_vertex_id} to {target_vertex_id} added with attributes {attributes}"
        logs_modif_network +=message
        return logs_modif_network


def erase_edge(source_vertex_id, target_vertex_id, vertex_type = "Nodes", edgeType = "distribute_to"):
    conn.delEdges(sourceVertexType=vertex_type,
                    targetVertexType=vertex_type,
                    edgeType=edgeType,
                    sourceVertexId=source_vertex_id,
                    targetVertexId=target_vertex_id)
    message =  f"Edge  from {source_vertex_id} to {target_vertex_id} deleted \n"
    logs_modif_network +=message
    return logs_modif_network

    


def erase_node(id ,vertex_type = "Nodes"):
    conn.delVerticesById(vertexIds=id,vertexType=vertex_type)
    message =  f"Node {id} deleted"
    logs_modif_network +=message
    return logs_modif_network

# Function Tab 3 (algorithms)

In [84]:
def launch_instaled_algo(algorithm,params_str):
    params = json.loads(params_str)
    results = conn.runInstalledQuery(algorithm, params= params)
    return results

In [85]:
VertexTypes = conn.getVertexTypes()
EdgeTypes = conn.getEdgeTypes()
EdgeAtributes = conn.getEdgeAttrs("distribute_to")[0][0]
data_algo = conn.getInstalledQueries(fmt="py")
installed_algorithms = [key.split("/")[-1] for key in data_algo.keys() if key.startswith("GET /query/VWG/")]

# Tab 4 

In [86]:
import matplotlib.pyplot as plt
import networkx as nx
from io import BytesIO
from PIL import Image

def networkx_maxflow(data):
    data = eval(data)
    max_flow = data[0]['@@sum_max_flow']
    edge_set = data[1]['@@edges_set']

    G = nx.DiGraph()

    # Add edges with capacities
    for edge in edge_set:
        from_node = edge['from_id']
        to_node = edge['to_id']
        capacity = edge['attributes']['Capacity']
        G.add_edge(from_node, to_node, capacity=capacity)

    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='lightblue', font_size=10, font_weight='bold')  # Draw nodes
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(u, v): f"{d['capacity']}" for u, v, d in G.edges(data=True)}, font_color='red')  # Edge labels
    plt.title(f"Max Flow: {max_flow}")
    plt.show()

    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)
    return img

In [87]:
import matplotlib.pyplot as plt
import networkx as nx
def networkx_astar(data):
    data = eval(data)
    node_data = {item['v_id']: item['attributes'] for item in data[2]['tmp']}
    edge_data = [{'from': edge['from_id'], 'to': edge['to_id'], 'capacity': edge['attributes']['Capacity']} for edge in data[2]['@@display_edge_set']]

    G = nx.Graph()

    # Add nodes
    for node_id, attributes in node_data.items():
        G.add_node(node_id, **attributes)

    # Add edges
    for edge in edge_data:
        G.add_edge(edge['from'], edge['to'], capacity=edge['capacity'])

    pos = nx.spring_layout(G)  # Nodes
    nx.draw(G, pos, with_labels=True, node_size=700, node_color='lightblue', font_size=10, font_weight='bold')  # Draw Edges
    nx.draw_networkx_edge_labels(G, pos, edge_labels={(edge['from'], edge['to']): f"{edge['capacity']}" for edge in edge_data}, font_color='red')  # Edge label
    plt.title("Graphe avec NetworkX")
    plt.show()
    from io import BytesIO
    from PIL import Image
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img = Image.open(buf)
    return img

In [88]:
def read_csv_path(input_file):
    df = pd.read_csv(input_file.name)
    target_column = sorted(list(df.columns))
    return  gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column) , gr.Dropdown(choices = target_column)

In [1]:
import gradio as gr
atributes_nodes = {}
atributes_edges = {}

class GradioMetro:
    
    def front_func(self):
        with gr.Blocks(css="Gradio/styles.css", title ="Graphs") as demo:
            gr.Markdown("## ""![](file/Gradio/logo_lis.svg) Graphs""", elem_classes="cabecero")
            with gr.Tabs() as tabs:
                with gr.TabItem("Cargar movimientos",id=0):
                    with gr.Row():
                        with gr.Column(scale=1):
                            upload_file = gr.File(label="Cargar fichero", type="filepath")
                        with gr.Column():
                            dropdown_column_start = gr.Dropdown(label='Columna del envio', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_arrival = gr.Dropdown(label='Columna de la llegada', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            dropdown_column_quantity = gr.Dropdown(label='Columna de la cantidad', choices = [],  interactive=True, allow_custom_value= True, elem_classes="dropdown")
                            load_data_send = gr.Button("Lanzar los envios")
                            upload_file.upload(read_csv_path,inputs=[upload_file],outputs=[dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity])
                            logs = gr.TextArea(label="Logs",lines=10)
                            load_data_send.click(make_daily_movements,inputs=[upload_file,dropdown_column_start,dropdown_column_arrival,dropdown_column_quantity], outputs=[logs])
                
                
                with gr.TabItem("KPIs movimiento",id=1):
                    with gr.Row():
                        with gr.Column(scale=10):
                            gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="800" height="800"
                                    src=""" + tigergraph_insights_map + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""")   
                        with gr.Column():
                            operation_number = gr.Number(label= "KPI : Numero de operaciones")
                            palets_number = gr.Number(label= "KPI : Numero de palets")
                            origin_number = gr.Number(label= "KPI : Numero de origen")
                            distinct_destinations = gr.Number(label= "KPI : destinaciones diferentes")
                            total_cost = gr.Number(label= "KPI : Coste total")
                            total_cost_per_palet = gr.Number(label= "KPI : Coste por palet")
                            button = gr.Button("Calcular KPIs")
                            button.click(get_KPIs_movements , outputs= [operation_number,palets_number,origin_number,distinct_destinations,total_cost,total_cost_per_palet]) 


                with gr.TabItem("KPIs red",id=2):
                    with gr.Row():
                        with gr.Column(scale=10):
                            gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="800" height="800"
                                    src=""" + tigergraph_insights_map + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""")   
                        with gr.Column():
                            Numedges = gr.Number(label="KPI : Cantidad de Aristas")
                            Numnodes = gr.Number(label="KPI : Cantidad de Nodos")
                            average_cost_per_edge = gr.Number(label= "KPI : Coste medio / Arista")
                            capacidad_total_aristas = gr.Number(label="KPI : Capacidad total de las aristas ")
                            warehouse_total_capacity = gr.Number(label= "KPI : Capacidad total de los Almacenes")
                            warehouse_total_stock = gr.Number(label="KPI : Stock total de los almacenes")
                            button = gr.Button("Calcular KPIs")
                            button.click(get_KPIs_network , outputs= [Numedges,Numnodes,average_cost_per_edge,capacidad_total_aristas,warehouse_total_capacity,warehouse_total_stock]) 
                
                with gr.TabItem("add nodes", id=3):
                    new_node_id = gr.Textbox(label="Node ID")
                    for e in attributes_nodes_list:
                        with gr.Row():
                            atributes_nodes[e] = gr.Number(label=e)
                
                    with gr.Row():
                        btn = gr.Button("Add Node")
                        output = gr.Textbox()

                    inputs = [new_node_id] + [atributes_nodes[e] for e in attributes_nodes_list]
                    btn.click(fn=add_node_with_values, inputs=inputs, outputs=output)


                with gr.TabItem("add edges", id=4):
                    source_vertex_id = gr.Textbox(label="Source Vertex ID")
                    target_vertex_id = gr.Textbox(label="Target Vertex ID")

                    for e in attributes_edges_list:
                        with gr.Row():
                            atributes_edges[e] = gr.Number(label=e)

                    with gr.Row():
                        btn = gr.Button("Add Edge")
                        output = gr.Textbox()

                        inputs = [
                            source_vertex_id,
                            target_vertex_id
                        ] + [atributes_edges[e] for e in attributes_edges_list]
                        btn.click(fn=add_edge_with_values, inputs=inputs, outputs=output)

                with gr.TabItem("Modificación red-arrista",id=5):
                    with gr.Row():

                        with gr.Column():   #ERASE 
                            source_node_id = gr.Dropdown(list_name_nodes,label='Punto de origen')
                            target_node_id = gr.Dropdown(list_name_nodes,label='Punto de destinacion')

                        with gr.Column():   #ADD
                            attributes_edges_list_gradio = gr.Dropdown(attributes_edges_list,label='Atributos aristas')
                            atributes_dict = gr.Number(label='Valor')
                            button_add_edge = gr.Button("Añadir/modificar arista")
                            button_add_edge.click(add_new_edge ,inputs = [source_node_id,target_node_id,atributes_dict]) 

                            button_erase_edge = gr.Button("Borrar arista")

                        with gr.Column():
                            logs = gr.TextArea(label="Logs",lines=10)
                            button_erase_edge.click(erase_edge, inputs = [source_node_id,target_node_id])
                            
                with gr.TabItem("KPIs movimiento alternativo",id=6):
                    with gr.Row():
                        with gr.Column(scale=10):
                            gr.HTML(value = """ <html lang="en"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, 
                                    initial-scale=1.0"></head><body><iframe width="800" height="800"
                                    src=""" + tigergraph_insights_map + """ 
                                    title="tigergarph insights" frameborder="0"></iframe></body></html>""")   
                        with gr.Column():
                            path_1 = gr.TextArea(label="Camino alternativo 1",lines=3)
                            path_2 = gr.TextArea(label="Camino alternativo 2",lines=3)
                            price_1 = gr.TextArea(label="KPI : Precio camino alternativo 1")
                            price_2 = gr.TextArea(label="KPI : Precio camino alternativo 2") 
                            button = gr.Button("Calcular KPIs")
                            button.click(get_KPIs_alternatives , outputs= [path_1,price_1,path_2,price_2]) 


                demo.launch(max_threads=1000)
                
if __name__ == "__main__":

    lis = GradioMetro()
    
    lis.front_func()

NameError: name 'read_csv_path' is not defined

IMPORTANT: You are using gradio version 4.27.0, however version 4.29.0 is available, please upgrade.
--------
